In [2]:
import pandas as pd
import numpy as np
from tpulse import TinkoffPulse
from time import sleep
from pytz import timezone
from httpx import HTTPStatusError
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import re

import time
from requests.exceptions import ReadTimeout

import warnings

warnings.filterwarnings("ignore")

In [ ]:

def parsing_tpulse(ticker, N, KEYS):

    cursor = None
    raw_data = []

    for _ in range(N):
        try:
            response = pulse.get_posts_by_ticker(ticker, cursor)
            try:
                cursor = response["nextCursor"]
            except:
                print(
                    f"Страниц с постами акции {ticker} меньше {N} - были выгружены все имеющиеся"
                )
                break
            posts = response["items"]
            for post in posts:
                data = {key: post[key] for key in KEYS}
                data["ticker"] = ticker
                data["text"] = post["content"]["text"]
                data["reactionCount"] = post["reactions"]["totalCount"]
                data["reactions_counters"] = post["reactions"]["counters"]
                raw_data.append(data)

        except HTTPStatusError:
            pass
        sleep(0.25)

    result_df = pd.DataFrame(raw_data)
    result_df["inserted"] = pd.to_datetime(result_df["inserted"]).dt.date

    return result_df


pulse = TinkoffPulse()

keys = ["id", "inserted", "commentsCount"]
share_list = [
    "SBER", "GAZP", "T", "X5", "PIKK", "PLZL", "LKOH", "GMKN", "VTBR", "NVTK",
    "YDEX", "TATN", "SMLT", "SPBE", "ROSN", "AFLT", "IRAO", "VKCO", "CHMF",
    "NLMK", "MAGN", "MGNT", "MTLR", "AFKS", "MOEX", "POSI", "ALRS", "BELU",
    "MTSS", "SVCB", "SNGS", "HEAD", "CNRU", "PHOR", "RTKM", "SIBN", "UGLD",
    "RUAL", "HYDR", "BSPB", "FEES", "EUTR", "RNFT", "MRKC", "MDMG", "FLOT",
    "UPRO", "ASTR", "IVAT", "LENT", "WUSH", "UWGN", "ENPG", "TRMK", "MVID",
    "RASP", "RAGR", "MRKP", "SVAV", "AQUA", "RENI", "SOFL", "SGZH", "SFIN",
    "OZPH", "NMTP", "FIXR", "OGKB", "SELG", "LEAS", "CBOM", "VSMO", "IRKT",
    "VSEH", "AKRN", "LSRG", "RTKMP", "SBERP", "SNGSP", "TATNP", "TTLK"
]
result_df = pd.DataFrame(
    columns=[
        "id",
        "inserted",
        "commentsCount",
        "text",
        "reactionCount",
        "reactions_counters",
    ]
)

max_retries, retry_delay, page_to_read, page_to_read_global = 20, 10, 20000, 20000

for share in share_list:
    page_to_read = page_to_read_global
    for attempt in range(1, max_retries + 1):
        try:
            result = parsing_tpulse(share, page_to_read, keys)
        except Exception as e:
            print(
                f"""
При работе с акцией {share} была получена ошибка: {e}.Попытка {attempt} из {max_retries}: таймаут. Ожидаем {retry_delay} секунд перед повтором. Сократим количество страниц по данной акции до {page_to_read // 2}
                """
            )
            page_to_read = round(page_to_read / 1.5)
            time.sleep(retry_delay)
        else:
            print(
                f"Данные по акции {share} выгружены в размере {result.shape} начиная с {min(result['inserted'])}\n"
            )
            result_df = pd.concat([result_df, result], axis=0)
            break
    else:
        print(
            "Превышено максимальное число попыток. Проверьте сеть и доступность сервера."
        )

result_df


Страниц с постами акции POSI меньше 20000 - были выгружены все имеющиеся
Данные по акции POSI выгружены в размере (50783, 7) начиная с 2021-12-15

Страниц с постами акции SGZH меньше 20000 - были выгружены все имеющиеся
Данные по акции SGZH выгружены в размере (85369, 7) начиная с 2021-04-24

Страниц с постами акции SMLT меньше 20000 - были выгружены все имеющиеся
Данные по акции SMLT выгружены в размере (16230, 7) начиная с 2020-11-02

Страниц с постами акции SPBE меньше 20000 - были выгружены все имеющиеся
Данные по акции SPBE выгружены в размере (79112, 7) начиная с 2021-11-15


При работе с акцией T была получена ошибка: The read operation timed out.Попытка 1 из 20: таймаут. Ожидаем 10 секунд перед повтором. Сократим количество страниц по данной акции до 10000
                

При работе с акцией T была получена ошибка: The read operation timed out.Попытка 2 из 20: таймаут. Ожидаем 10 секунд перед повтором. Сократим количество страниц по данной акции до 6666
                
Стран

,id,inserted,commentsCount,text,reactionCount,reactions_counters,ticker
0,d231a646-7fe8-4689-b7f9-d2a001a6a630,2025-10-31,0,Позитив Technologies: когда акции растут быстр...,6,"[{'type': 'like', 'count': 5}, {'type': 'rocke...",POSI
1,367fe19d-a208-47a3-b522-96ff06c02a5e,2025-10-31,0,"🌟 Доброе утро!\nДайджест на пятницу, 31 октябр...",11,"[{'type': 'like', 'count': 8}, {'type': 'buy-u...",POSI
2,1384d5f4-74d4-46ef-9562-b6d0dcd7c8b2,2025-10-31,1,"🥳 Доброе утро друзья, пятница 🍁\n\n🚨 Мысли по ...",17,"[{'type': 'like', 'count': 15}, {'type': 'buy-...",POSI
3,b3c725cc-8294-4f71-ae2c-507f26300e4c,2025-10-30,0,Закрытие дня осноаная сессия {$MOEX} : \n\nИн...,1,"[{'type': 'rocket', 'count': 1}]",POSI
4,dacf2b2b-8330-4b11-85b3-352a537b43dd,2025-10-30,15,​​⚡​​ИНТЕРЕСНЫЙ РАСКЛАД ⚡ #новости \n30.10.202...,112,"[{'type': 'like', 'count': 108}, {'type': 'buy...",POSI
...,...,...,...,...,...,...,...
11833,c2fcdd7a-117a-4ccc-90ef-94f4e3c6890e,2024-07-09,18,Всем прибыли и счастья! \n{$ROSN} \nЛюбимая не...,16,"[{'type': 'like', 'count': 10}, {'type': 'buy-...",YDEX
11834,e383df42-509d-4b10-842b-74e62c0e6dda,2024-07-09,3,🇷🇺 НОВОСТИ И МНЕНИЯ ДНЯ 👇 \n\n📌 {$GAZP} \nБулг...,10,"[{'type': 'like', 'count': 6}, {'type': 'buy-u...",YDEX
11835,0412da43-2ebd-4daf-ad2d-7f543e7dbc07,2024-07-09,1,ОЖИДАЕМЫЕ СОБЫТИЯ НА СЕГОДНЯ\n________________...,8,"[{'type': 'like', 'count': 6}, {'type': 'rocke...",YDEX
11836,1ed8c5d6-3b02-4c3a-a84c-75eadef39302,2024-07-09,19,"Минус Лям за месяц. Ухожу с рынка…😕\n\nАга, ко...",61,"[{'type': 'like', 'count': 49}, {'type': 'buy-...",YDEX
